In [1]:
import numpy as np
import pandas as pd
import warnings # 경고 메시지 무시
warnings.filterwarnings(action='ignore')
from konlpy.tag import Mecab # 형태소 분석기
mecab  = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식 패키지
from gensim import corpora # 단어 빈도수 계산 패키지
import gensim # LDA 모델 활용 목적
import pyLDAvis.gensim_models # LDA 시각화용 패키지
from collections import Counter # 단어 등장 횟수 카운트

In [2]:
news_taxi = pd.read_excel("D:/issue/data/news/taxi/taxi_OR.xlsx")
news_taxi = news_taxi.loc[:, ['제목', '본문']]
news_taxi = news_taxi.drop_duplicates()
news_taxi

,제목,본문
0,"[민심으로 보는 세상] 타다 우버, 차량 공유 부활? “찬성 83%”",그야말로 ‘함흥차사’다. 지난해 말부터 서울 수도권을 중심으로 발발한 ‘택시 대란’...
1,[사설] '기득권 사수'엔 유능하고 '혁신'엔 무능한 정치인,차량 호출 서비스 ‘타다’를 운영하다 불법 콜택시 영업 혐의로 기소된 이재웅 전 쏘...
2,'극적 타결' 버스대란 피했지만 파업 불씨 여전,【앵커】 \n\n경기도 버스 노사 협상이 극적으로 타결되면서 걱정했던 버스 대란은 ...
3,"3년만에 LG사장단 한자리 모였다 구광모 회장, 당부한 말이","""A사장님, 이 회사의 제품이 왜 고객에게 좋은 반응을 얻었다고 생각하시나요? 이 ..."
4,"추락한 손정의 비전펀드, 직원 30% 줄인다",손정의(孫正義 일본명 손마사요시) 회장이 이끄는 일본 소프트뱅크그룹이 자사가 운용하...
...,...,...
2615,"UT, 7일까지 통합 앱 '베타테스터' 모집...택시 30% 할인 혜택",우버와 티맵모빌리티의 합작회사 우티 유한회사(UT LLC)는 오는 7일까지 연내 출...
2616,"우티(UT), 우버와 앱 통합 앞두고 베타테스터 모집 시작",우버와 티맵모빌리티의 합작회사 우티 유한회사(UT LLC)는 연내 통합 애플리케이션...
2617,"""전북 버스대란 피했다""...임실 순창 남원 제외 모두 복귀","전북 전주시를 제외한 13개 시 군 버스 기사들이 1일 첫차부터 파업에 나섰지만, ..."
2618,"""카카오 잡다 혁신 다 죽일라"" '플랫폼 국감' 앞두고 경계 목소리","카카오의 ‘문어발 확장’에 대한 정부와 정치권의 압박이 거센 가운데, 국정감사를 계..."


In [3]:
news_taxi.isnull().sum()

제목    0
본문    0
dtype: int64

In [4]:
news_taxi_removed = list(map(lambda news_taxi: re.sub('[^가-힣 ]', '', news_taxi), news_taxi['본문']))
news_taxi['본문'] = news_taxi_removed
news_taxi

,제목,본문
0,"[민심으로 보는 세상] 타다 우버, 차량 공유 부활? “찬성 83%”",그야말로 함흥차사다 지난해 말부터 서울 수도권을 중심으로 발발한 택시 대란이 잡힐 ...
1,[사설] '기득권 사수'엔 유능하고 '혁신'엔 무능한 정치인,차량 호출 서비스 타다를 운영하다 불법 콜택시 영업 혐의로 기소된 이재웅 전 쏘카 ...
2,'극적 타결' 버스대란 피했지만 파업 불씨 여전,앵커 경기도 버스 노사 협상이 극적으로 타결되면서 걱정했던 버스 대란은 피했습니다 ...
3,"3년만에 LG사장단 한자리 모였다 구광모 회장, 당부한 말이",사장님 이 회사의 제품이 왜 고객에게 좋은 반응을 얻었다고 생각하시나요 이 기업이 ...
4,"추락한 손정의 비전펀드, 직원 30% 줄인다",손정의 일본명 손마사요시 회장이 이끄는 일본 소프트뱅크그룹이 자사가 운용하는 비전펀...
...,...,...
2615,"UT, 7일까지 통합 앱 '베타테스터' 모집...택시 30% 할인 혜택",우버와 티맵모빌리티의 합작회사 우티 유한회사 는 오는 일까지 연내 출시할 통합 앱의...
2616,"우티(UT), 우버와 앱 통합 앞두고 베타테스터 모집 시작",우버와 티맵모빌리티의 합작회사 우티 유한회사 는 연내 통합 애플리케이션앱 출시를 앞...
2617,"""전북 버스대란 피했다""...임실 순창 남원 제외 모두 복귀",전북 전주시를 제외한 개 시 군 버스 기사들이 일 첫차부터 파업에 나섰지만 곳을 제...
2618,"""카카오 잡다 혁신 다 죽일라"" '플랫폼 국감' 앞두고 경계 목소리",카카오의 문어발 확장에 대한 정부와 정치권의 압박이 거센 가운데 국정감사를 계기로 ...


In [5]:
replace_list = pd.read_excel("D:/issue/data/news/taxi/replace_list.xlsx")
replace_list.head()

,before_replacement,after_replacement
0,앱,어플


In [6]:
def replace_word(news):
    for i in range(len(replace_list['before_replacement'])):
        try:
            # 치환할 단어가 있는 경우에만 데이터 치환 수행
            if replace_list['before_replacement'][i] in news:
                news = news.replace(replace_list['before_replacement'][i], replace_list['after_replacement'][i])
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")
    return news

In [7]:
news_taxi['content'] = ''
news_replaced_list = []
for news in tqdm(news_taxi['본문']):
    news_replaced = replace_word(news) # 문자열 데이터 치환
    news_replaced_list.append(news_replaced)
    
news_taxi['content'] = news_replaced_list
news_taxi = news_taxi.loc[:, ['제목', 'content']]
news_taxi.head()

100%|██████████| 2598/2598 [00:00<00:00, 156215.35it/s]


,제목,content
0,"[민심으로 보는 세상] 타다 우버, 차량 공유 부활? “찬성 83%”",그야말로 함흥차사다 지난해 말부터 서울 수도권을 중심으로 발발한 택시 대란이 잡힐 ...
1,[사설] '기득권 사수'엔 유능하고 '혁신'엔 무능한 정치인,차량 호출 서비스 타다를 운영하다 불법 콜택시 영업 혐의로 기소된 이재웅 전 쏘카 ...
2,'극적 타결' 버스대란 피했지만 파업 불씨 여전,앵커 경기도 버스 노사 협상이 극적으로 타결되면서 걱정했던 버스 대란은 피했습니다 ...
3,"3년만에 LG사장단 한자리 모였다 구광모 회장, 당부한 말이",사장님 이 회사의 제품이 왜 고객에게 좋은 반응을 얻었다고 생각하시나요 이 기업이 ...
4,"추락한 손정의 비전펀드, 직원 30% 줄인다",손정의 일본명 손마사요시 회장이 이끄는 일본 소프트뱅크그룹이 자사가 운용하는 비전펀...


In [8]:
news_taxi_token = list(map(lambda news_taxi: mecab.nouns(news_taxi), news_taxi['content'])) # 토큰화(형태소 분석)
news_taxi_token

[['함흥차사',
  '지난해',
  '말',
  '서울',
  '수도',
  '중심',
  '발발',
  '택시',
  '대란',
  '기미',
  '택시',
  '데',
  '실패',
  '시민',
  '스마트폰',
  '화면',
  '시간',
  '길바닥',
  '허비',
  '택시',
  '대중교통',
  '전',
  '시간',
  '저녁',
  '회식',
  '자리',
  '양상',
  '포스트',
  '코로나',
  '시대',
  '심야',
  '매출',
  '회'],
 ['차량',
  '호출',
  '서비스',
  '운영',
  '불법',
  '콜택시',
  '영업',
  '혐의',
  '기소',
  '이재웅',
  '쏘카',
  '대표',
  '항소심',
  '무죄',
  '판결',
  '심',
  '마찬가지',
  '여객',
  '자동차',
  '운수업',
  '기사',
  '알선',
  '포함',
  '단기',
  '승합차',
  '대여',
  '렌터카',
  '계약',
  '논지',
  '년',
  '월',
  '스마트폰',
  '어플',
  '운전기사',
  '인승',
  '승합차',
  '호출'],
 ['앵커',
  '경기도',
  '버스',
  '노사',
  '협상',
  '극',
  '타결',
  '걱정',
  '버스',
  '대란',
  '임기',
  '안',
  '준공',
  '영제',
  '전면',
  '시행',
  '김동연',
  '도지사',
  '약속',
  '역할',
  '파업',
  '불씨',
  '건',
  '이여진',
  '기자',
  '앵커',
  '경기도',
  '버스',
  '노사',
  '협상',
  '타결',
  '노조',
  '파업',
  '철회',
  '협상',
  '시한',
  '오늘',
  '시'],
 ['사장',
  '회사',
  '제품',
  '고객',
  '반응',
  '생각',
  '기업',
  '과정',
  '고객',
  '경험',
  '혁신',
  '일',
  '경기도',
  '광주시',
 

In [9]:
# 한 글자이거나 '택시'인 단어 제거
news_taxi_token_removed = []
for i, v in enumerate(news_taxi_token):
#     print('news_taxi_token > ', i, v)
    line = []
    for j, k in enumerate(v):
        if not ((len(k) == 1) | (k == '택시')):
#             print('===>', j, k)
            line.append(news_taxi_token[i][j])
    news_taxi_token_removed.append(line)

In [10]:
news_taxi_token_removed

[['함흥차사',
  '지난해',
  '서울',
  '수도',
  '중심',
  '발발',
  '대란',
  '기미',
  '실패',
  '시민',
  '스마트폰',
  '화면',
  '시간',
  '길바닥',
  '허비',
  '대중교통',
  '시간',
  '저녁',
  '회식',
  '자리',
  '양상',
  '포스트',
  '코로나',
  '시대',
  '심야',
  '매출'],
 ['차량',
  '호출',
  '서비스',
  '운영',
  '불법',
  '콜택시',
  '영업',
  '혐의',
  '기소',
  '이재웅',
  '쏘카',
  '대표',
  '항소심',
  '무죄',
  '판결',
  '마찬가지',
  '여객',
  '자동차',
  '운수업',
  '기사',
  '알선',
  '포함',
  '단기',
  '승합차',
  '대여',
  '렌터카',
  '계약',
  '논지',
  '스마트폰',
  '어플',
  '운전기사',
  '인승',
  '승합차',
  '호출'],
 ['앵커',
  '경기도',
  '버스',
  '노사',
  '협상',
  '타결',
  '걱정',
  '버스',
  '대란',
  '임기',
  '준공',
  '영제',
  '전면',
  '시행',
  '김동연',
  '도지사',
  '약속',
  '역할',
  '파업',
  '불씨',
  '이여진',
  '기자',
  '앵커',
  '경기도',
  '버스',
  '노사',
  '협상',
  '타결',
  '노조',
  '파업',
  '철회',
  '협상',
  '시한',
  '오늘'],
 ['사장',
  '회사',
  '제품',
  '고객',
  '반응',
  '생각',
  '기업',
  '과정',
  '고객',
  '경험',
  '혁신',
  '경기도',
  '광주시',
  '곤지암',
  '리조트',
  '코로나',
  '온라인',
  '진행',
  '그룹',
  '장단',
  '워크숍',
  '주재우',
  '국민대',
  '경영',
  '학부',
  '교수'

In [11]:
NUM_TOPICS = 3
PASSES = 20

In [12]:
def lda_modeling(news_taxi_token):
    # 단어 인코딩 및 빈도수 계산
    dictionary = corpora.Dictionary(news_taxi_token)
    corpus = [dictionary.doc2bow(token) for token in news_taxi_token]
    # LDA 모델 학습
    model = gensim.models.ldamodel.LdaModel(corpus, 
                                            num_topics = NUM_TOPICS, 
                                            id2word = dictionary, 
                                            passes = PASSES)
    return model, corpus, dictionary

In [13]:
def print_topic_prop(topics, RATING):
    # 토픽에 따른 단어와 값 엑셀로 저장
    topic_values = []
    for topic in topics:
        topic_value = topic[1]
        topic_values.append(topic_value)
    topic_prop = pd.DataFrame({"topic_num" : list(range(1, NUM_TOPICS + 1)), "word_prop": topic_values})
    topic_prop.to_excel('D:/issue/taxi/result/topic_prop_' + RATING +  '.xlsx')
    display(topic_prop)

In [14]:
def lda_visualize(model, corpus, dictionary, RATING):
    pyLDAvis.enable_notebook()
    result_visualized = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    pyLDAvis.display(result_visualized)
    # 시각화 결과 저장
    RESULT_FILE = 'D:/issue/taxi/result/lda_result_' + RATING + '.html'
    pyLDAvis.save_html(result_visualized, RESULT_FILE)

In [15]:
model, corpus, dictionary = lda_modeling(news_taxi_token_removed)
NUM_WORDS = 10

In [16]:
RATING = 'taxi_OR'
topics = model.print_topics(num_words = NUM_WORDS)
print_topic_prop(topics, RATING)

,topic_num,word_prop
0,1,"0.021*""우버"" + 0.018*""미국"" + 0.017*""모빌리티"" + 0.013..."
1,2,"0.014*""기업"" + 0.010*""투자"" + 0.007*""경제"" + 0.007*""..."
2,3,"0.014*""서울"" + 0.012*""심야"" + 0.012*""시간"" + 0.010*""..."


In [17]:
lda_visualize(model, corpus, dictionary, RATING)

In [18]:
# 0에 가까울수록 전체 문서에서 출현 횟수는 적더라도 해당 토픽을 다른 토픽과 차별성 있게 구분할 수 있는 단어
# 1에 가까울수록 특정 토픽을 구성하는 키워드보다는 전체 문서 데이터에서 얼마나 빈출 되는 키워드일 가능성이 높음